# EV Charging Prediction Demo: Two-Stage Pipeline

**Purpose:** Interactive demonstration of the trained pipeline for predicting EV charging session durations.

**What This Does:**
1. Load trained Stage 1 classifier and Stage 2 regressor
2. Present real example sessions from the dataset
3. Show two-stage predictions with probabilities
4. Visualize decision confidence and duration estimates

In [11]:
# Import Required Libraries
import os, warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print('✓ Libraries imported successfully')

✓ Libraries imported successfully


In [12]:
# Load Data
csv_path = 'data/ev_sessions_clean.csv'
df = pd.read_csv(csv_path)
df['Start_plugin_dt'] = pd.to_datetime(df['Start_plugin_dt'])
df = df.sort_values('Start_plugin_dt').reset_index(drop=True)

# Chronological split (same as pipeline notebook)
split_idx = int(len(df) * 0.8)
train_df = df.iloc[:split_idx].copy()
test_df = df.iloc[split_idx:].copy()

print(f'✓ Data loaded: {len(df)} total sessions')
print(f'  Train: {len(train_df)} | Test: {len(test_df)}')

✓ Data loaded: 6745 total sessions
  Train: 5396 | Test: 1349


---
## Preparation: Re-Train Models

In [13]:
# Rebuild aggregates and features (same logic as pipeline)
def build_aggregates(train_df):
    user_agg = train_df.groupby('User_ID').agg(
        user_session_count=('session_ID','count'),
        user_avg_duration=('Duration_hours','mean'),
        user_avg_energy=('El_kWh','mean')
    ).reset_index()
    gar_agg = train_df.groupby('Garage_ID').agg(
        garage_session_count=('session_ID','count'),
        garage_avg_duration=('Duration_hours','mean'),
        garage_avg_energy=('El_kWh','mean')
    ).reset_index()
    return user_agg, gar_agg

def merge_aggregates(df_in, user_agg, gar_agg, train_df):
    df_m = df_in.merge(user_agg, on='User_ID', how='left').merge(gar_agg, on='Garage_ID', how='left')
    df_m['user_session_count'] = df_m['user_session_count'].fillna(0)
    df_m['garage_session_count'] = df_m['garage_session_count'].fillna(0)
    dur_mean = train_df['Duration_hours'].mean()
    eng_mean = train_df['El_kWh'].mean()
    df_m['user_avg_duration'] = df_m['user_avg_duration'].fillna(dur_mean)
    df_m['garage_avg_duration'] = df_m['garage_avg_duration'].fillna(dur_mean)
    df_m['user_avg_energy'] = df_m['user_avg_energy'].fillna(eng_mean)
    df_m['garage_avg_energy'] = df_m['garage_avg_energy'].fillna(eng_mean)
    return df_m

user_agg, gar_agg = build_aggregates(train_df)
train_enh = merge_aggregates(train_df, user_agg, gar_agg, train_df)
test_enh = merge_aggregates(test_df, user_agg, gar_agg, train_df)

# Define features
base_num = ['hour_sin','hour_cos','temp','precip','wind_spd','clouds','solar_rad']
weather_flags = ['is_rainy','is_overcast','is_sunny']
agg_num = ['user_session_count','user_avg_duration','user_avg_energy',
           'garage_session_count','garage_avg_duration','garage_avg_energy']
base_cat = ['weekday','Garage_ID','month_plugin']

num_features = base_num + weather_flags + agg_num
cat_features = base_cat

print('✓ Features and aggregates prepared')

✓ Features and aggregates prepared


In [14]:
# Re-train Stage 1 classifier
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support

preprocessor_cls = ColumnTransformer([
    ('num', StandardScaler(), num_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
])

X_train_cls = train_enh[num_features + cat_features]
y_train_cls = (1 - train_enh['is_short_session']).astype(int)

X_train_p = preprocessor_cls.fit_transform(X_train_cls)
X_train_dense = X_train_p.toarray() if hasattr(X_train_p, 'toarray') else X_train_p

# Train classifier with sample weights
scale_pos = (1 - y_train_cls.mean()) / y_train_cls.mean()
sample_weights = np.where(y_train_cls == 1, scale_pos, 1.0)

clf = HistGradientBoostingClassifier(
    max_iter=300, max_depth=6, learning_rate=0.05,
    early_stopping=True, n_iter_no_change=20, random_state=42, verbose=0
)

clf.fit(X_train_dense, y_train_cls, sample_weight=sample_weights)

# Get optimal threshold from training
proba_long_train = clf.predict_proba(X_train_dense)[:, 1]
thresholds = np.linspace(0.1, 0.9, 40)
best_f1 = -1
optimal_threshold = 0.5

for t in thresholds:
    y_pred = (proba_long_train >= t).astype(int)
    _, _, f1, _ = precision_recall_fscore_support(y_train_cls, y_pred, average='binary', zero_division=0)
    if f1 > best_f1:
        best_f1 = f1
        optimal_threshold = t

print(f'✓ Stage 1 Classifier trained and ready')
print(f'  Optimal threshold: {optimal_threshold:.3f}')

✓ Stage 1 Classifier trained and ready
  Optimal threshold: 0.674


In [15]:
# Re-train Stage 2 regressor
train_short = train_enh[train_enh['Duration_hours'] < 24].copy()
X_train_reg = train_short[num_features + cat_features]
y_train_reg = np.log1p(train_short['Duration_hours'].values)

preprocessor_reg = ColumnTransformer([
    ('num', StandardScaler(), num_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
])

rf_reg = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1, min_samples_leaf=2)
rf_pipe = Pipeline([('prep', preprocessor_reg), ('rf', rf_reg)])
rf_pipe.fit(X_train_reg, y_train_reg)

print(f'✓ Stage 2 Regressor trained on {len(train_short)} short sessions')

✓ Stage 2 Regressor trained on 5049 short sessions


---
## Interactive Demo: Make Predictions on Real Test Sessions

In [21]:
def predict_session(session_idx, df_enhanced, clf, preprocessor_cls, rf_pipe, num_features, cat_features, threshold):
    """
    Make two-stage predictions for a single session.
    - Ensures a DataFrame input to preprocessors so column names are preserved.
    """
    session = df_enhanced.iloc[session_idx]

    # Prepare features for Stage 1 as DataFrame (keeps column names for ColumnTransformer)
    sample_df = pd.DataFrame([session[num_features + cat_features]])
    X_proc = preprocessor_cls.transform(sample_df)
    X_dense = X_proc.toarray() if hasattr(X_proc, 'toarray') else X_proc

    # Stage 1 prediction
    proba_long = clf.predict_proba(X_dense)[0, 1]
    is_long_predicted = proba_long >= threshold

    result = {
        'session_id': session['session_ID'],
        'user_id': session['User_ID'],
        'garage': session['Garage_ID'],
        'start_time': session['Start_plugin_dt'],
        'actual_duration': session['Duration_hours'],
        'actual_is_long': session['Duration_hours'] >= 24,
        'stage1_prob_long': proba_long,
        'stage1_predicted_long': is_long_predicted,
    }

    # Stage 2 prediction (only if predicted short)
    if not is_long_predicted:
        X_reg = pd.DataFrame([session[num_features + cat_features]])
        y_pred_log = rf_pipe.predict(X_reg)[0]
        y_pred_hours = np.expm1(y_pred_log)
        result['stage2_pred_duration'] = y_pred_hours
    else:
        result['stage2_pred_duration'] = None

    return result

print('✓ Prediction function ready')

✓ Prediction function ready


### Pick a User (Dropdown) and Predict Latest Session
Use the dropdown to select any user from the test set. We will take their most recent session and run the two-stage pipeline.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Build dropdown of users in test set
user_options = sorted(test_enh['User_ID'].unique())
user_dropdown = widgets.Dropdown(options=user_options, description='User')

def predict_latest_for_user(user_id):
    # Safety: ensure models are available
    missing = [name for name in ['clf','rf_pipe','preprocessor_cls','num_features','cat_features','optimal_threshold'] if name not in globals()]
    if missing:
        clear_output(wait=True)
        display(user_dropdown)
        print(f"Models not ready. Missing: {', '.join(missing)}. Run the training cells above (Stage 1 and Stage 2).")
        return

    clear_output(wait=True)
    display(user_dropdown)
    user_rows = test_enh[test_enh['User_ID'] == user_id]
    if user_rows.empty:
        print(f'No test sessions for user {user_id}.')
        return
    session_row = user_rows.sort_values('Start_plugin_dt').iloc[-1]
    session_idx = session_row.name
    pred = predict_session(session_idx, test_enh, clf, preprocessor_cls, rf_pipe, num_features, cat_features, optimal_threshold)

    print('='*70)
    print(f"LATEST TEST SESSION FOR USER {user_id}")
    print(f"SESSION ID: {pred['session_id']} | Garage: {pred['garage']}")
    print(f"Start Time: {pred['start_time']}")
    print('='*70)
    print("\nCONTEXT (weather & timing):")
    print(f"  Temp: {session_row['temp']:.1f}C | Precip: {session_row['precip']:.2f} | Clouds: {session_row['clouds']:.0f}%")
    print(f"  Solar Radiation: {session_row['solar_rad']:.1f} | Wind: {session_row['wind_spd']:.1f}")
    print(f"  Weekday: {session_row['weekday']} | Hour: {session_row['hour_sin']:.2f}/{session_row['hour_cos']:.2f}")

    print("\nACTUAL OUTCOME:")
    print(f"  Duration: {pred['actual_duration']:.2f} hours")
    print(f"  Classification: {'LONG (>=24h)' if pred['actual_is_long'] else 'SHORT (<24h)'}")

    print("\nPREDICTION:")
    print(f"  STAGE 1: P(Long >=24h) = {pred['stage1_prob_long']:.1%} -> Decision: {'LONG' if pred['stage1_predicted_long'] else 'SHORT'} (threshold {optimal_threshold:.3f})")
    if pred['stage2_pred_duration'] is not None:
        print(f"  STAGE 2: Predicted duration = {pred['stage2_pred_duration']:.2f} hours")
        print(f"           Error vs actual = {abs(pred['actual_duration'] - pred['stage2_pred_duration']):.2f} hours")
    else:
        print("  STAGE 2: Skipped (predicted LONG)")

    print("\nNEXT ACTION FOR OPERATOR:")
    if pred['stage1_predicted_long']:
        print("  -> Reserve extended parking / slower turnover bay")
    else:
        print("  -> Keep standard bay; expect turnover within the predicted window")

display(user_dropdown)
widgets.interact(predict_latest_for_user, user_id=user_dropdown);

Dropdown(description='User', options=('AdA1-1', 'AdA6-1', 'AdO1-1', 'AdO1-2', 'AdO1-3', 'AdO1-4', 'AdO3-1', 'A…

interactive(children=(Dropdown(description='User', options=('AdA1-1', 'AdA6-1', 'AdO1-1', 'AdO1-2', 'AdO1-3', …

In [23]:
# Set a user_id and run a single prediction (no dropdown)
user_id = user_options[0]  # change this to any user in user_options

# Safety: ensure models are available
missing = [name for name in ['clf','rf_pipe','preprocessor_cls','num_features','cat_features','optimal_threshold'] if name not in globals()]
if missing:
    print(f"Models not ready. Missing: {', '.join(missing)}. Run the training cells above (Stage 1 and Stage 2).")
else:
    user_rows = test_enh[test_enh['User_ID'] == user_id]
    if user_rows.empty:
        print(f'No test sessions for user {user_id}.')
    else:
        session_row = user_rows.sort_values('Start_plugin_dt').iloc[-1]
        session_idx = session_row.name
        pred = predict_session(session_idx, test_enh, clf, preprocessor_cls, rf_pipe, num_features, cat_features, optimal_threshold)
        print(f"USER {user_id} | SESSION {pred['session_id']} | Garage {pred['garage']}")
        print(f"Start: {pred['start_time']}")
        print(f"Stage1 P(Long): {pred['stage1_prob_long']:.1%} -> {'LONG' if pred['stage1_predicted_long'] else 'SHORT'} (thr={optimal_threshold:.3f})")
        if pred['stage2_pred_duration'] is not None:
            print(f"Stage2 Pred Duration: {pred['stage2_pred_duration']:.2f}h | Actual: {pred['actual_duration']:.2f}h")
        else:
            print("Stage2 skipped (predicted LONG)")

USER AdA1-1 | SESSION 6713 | Garage AdA1
Start: 2020-01-27 20:22:00
Stage1 P(Long): 57.2% -> SHORT (thr=0.674)
Stage2 Pred Duration: 7.35h | Actual: 35.05h


In [ ]:
# Select diverse examples from test set
short_sessions = test_enh[test_enh['Duration_hours'] < 24].sort_values('Duration_hours')
example1_idx = short_sessions.index[len(short_sessions)//4]

long_sessions = test_enh[test_enh['Duration_hours'] >= 24].sort_values('Duration_hours')
example2_idx = long_sessions.index[len(long_sessions)//2]

example3_idx = short_sessions.index[len(short_sessions)//2]

# Make predictions
pred1 = predict_session(example1_idx, test_enh, clf, preprocessor_cls, rf_pipe, num_features, cat_features, optimal_threshold)
pred2 = predict_session(example2_idx, test_enh, clf, preprocessor_cls, rf_pipe, num_features, cat_features, optimal_threshold)
pred3 = predict_session(example3_idx, test_enh, clf, preprocessor_cls, rf_pipe, num_features, cat_features, optimal_threshold)

print('✓ Example predictions generated')

### Example 1: Short Session (Early Checkout)

In [ ]:
print('='*70)
print(f"SESSION ID: {pred1['session_id']} | User: {pred1['user_id']} | Garage: {pred1['garage']}")
print(f"Start Time: {pred1['start_time']}")
print('='*70)
print(f"\nACTUAL OUTCOME:")
print(f"  Duration: {pred1['actual_duration']:.2f} hours")
print(f"  Classification: {'LONG (≥24h)' if pred1['actual_is_long'] else 'SHORT (<24h)'}")
print(f"\nPREDICTION:")
print(f"\n  STAGE 1 (Classification):")
print(f"    Probability of Long (≥24h): {pred1['stage1_prob_long']:.1%}")
print(f"    Decision: {'LONG' if pred1['stage1_predicted_long'] else 'SHORT'}")
print(f"\n  STAGE 2 (Regression for Short):")
if pred1['stage2_pred_duration'] is not None:
    print(f"    Predicted Duration: {pred1['stage2_pred_duration']:.2f} hours")
    print(f"    Error: {abs(pred1['actual_duration'] - pred1['stage2_pred_duration']):.2f} hours")
else:
    print(f"    [Not applicable - classified as Long]")
print(f"\nCONCLUSION:")
if not pred1['stage1_predicted_long']:
    print(f"  ✓ Model predicts SHORT session ({pred1['stage1_prob_long']:.1%} confidence)")
    print(f"  ✓ Estimated {pred1['stage2_pred_duration']:.1f}h (actual: {pred1['actual_duration']:.1f}h)")
else:
    print(f"  ✗ Model predicted LONG, but actually SHORT")

### Example 2: Long Session (Extended Stay)

In [ ]:
print('='*70)
print(f"SESSION ID: {pred2['session_id']} | User: {pred2['user_id']} | Garage: {pred2['garage']}")
print(f"Start Time: {pred2['start_time']}")
print('='*70)
print(f"\nACTUAL OUTCOME:")
print(f"  Duration: {pred2['actual_duration']:.2f} hours")
print(f"  Classification: {'LONG (≥24h)' if pred2['actual_is_long'] else 'SHORT (<24h)'}")
print(f"\nPREDICTION:")
print(f"\n  STAGE 1 (Classification):")
print(f"    Probability of Long (≥24h): {pred2['stage1_prob_long']:.1%}")
print(f"    Decision: {'LONG' if pred2['stage1_predicted_long'] else 'SHORT'}")
print(f"\n  STAGE 2 (Regression for Short):")
if pred2['stage2_pred_duration'] is not None:
    print(f"    Predicted Duration: {pred2['stage2_pred_duration']:.2f} hours")
else:
    print(f"    [Not applicable - classified as Long]")
print(f"\nCONCLUSION:")
if pred2['stage1_predicted_long']:
    print(f"  ✓ Model predicts LONG session ({pred2['stage1_prob_long']:.1%} confidence)")
    print(f"  ✓ Grid operator should reserve extended parking space")
else:
    print(f"  ✗ Model predicted SHORT, but actually LONG (missed long session)")

### Example 3: Another Short Session (Medium Duration)

In [ ]:
print('='*70)
print(f"SESSION ID: {pred3['session_id']} | User: {pred3['user_id']} | Garage: {pred3['garage']}")
print(f"Start Time: {pred3['start_time']}")
print('='*70)
print(f"\nACTUAL OUTCOME:")
print(f"  Duration: {pred3['actual_duration']:.2f} hours")
print(f"  Classification: {'LONG (≥24h)' if pred3['actual_is_long'] else 'SHORT (<24h)'}")
print(f"\nPREDICTION:")
print(f"\n  STAGE 1 (Classification):")
print(f"    Probability of Long (≥24h): {pred3['stage1_prob_long']:.1%}")
print(f"    Decision: {'LONG' if pred3['stage1_predicted_long'] else 'SHORT'}")
print(f"\n  STAGE 2 (Regression for Short):")
if pred3['stage2_pred_duration'] is not None:
    print(f"    Predicted Duration: {pred3['stage2_pred_duration']:.2f} hours")
    print(f"    Error: {abs(pred3['actual_duration'] - pred3['stage2_pred_duration']):.2f} hours")
else:
    print(f"    [Not applicable - classified as Long]")
print(f"\nCONCLUSION:")
if not pred3['stage1_predicted_long']:
    print(f"  ✓ Model predicts SHORT session ({pred3['stage1_prob_long']:.1%} confidence)")
    print(f"  ✓ Estimated {pred3['stage2_pred_duration']:.1f}h (actual: {pred3['actual_duration']:.1f}h)")
else:
    print(f"  ✗ Model predicted LONG, but actually SHORT")

---
## Summary: How to Use in Production

In [ ]:
print("""\n" + "="*70 + """
PRODUCTION WORKFLOW
" + "="*70 + """

1. USER PLUGS IN CAR
   → Extract time, location, weather, user history

2. RUN STAGE 1 CLASSIFIER
   → Get P(Long ≥ 24h)
   → If P ≥ 0.633:
      Output: "LONG SESSION - Reserve extended parking"
   → Else: Proceed to Stage 2

3. RUN STAGE 2 REGRESSOR (if predicted short)
   → Get estimated duration
   → Output: "Expected ~X hours ±5h"
   → Use for charging schedule optimization

4. CONTINUOUSLY MONITOR
   → Track actual vs predicted outcomes
   → Retrain quarterly with new data
   → Adjust threshold if business needs change

" + "="*70 + """
KEY METRICS
" + "="*70 + """

Stage 1: Long-Session Detection
  • AUC: 0.847 (excellent discrimination)
  • Recall: 59% (identifies 59 of 105 long sessions)
  • Precision: 33.5% (of predicted long, 1/3 are correct)
  • Threshold: 0.633 (optimal F1)

Stage 2: Short-Duration Prediction
  • RMSE: 5.95 hours
  • MAE: 4.19 hours (typical error)
  • R²: 0.161 (explains 16% of variance)
  • Training on short-only prevents tail bias

Business Impact:
  • 29x improvement in long-session detection (59% vs 2%)
  • ±5 hour accuracy for charging schedule planning
  • Probabilistic output enables risk-based decisions
  • Modular pipeline scales easily

" + "="*70
""")